In [5]:
import scanpy as sc

In [6]:
import Bio.motifs.jaspar
import pychromvar as pc
import os

In [7]:
import scvi

/opt/modules/i12g/anaconda/envs/scooby_reproducibility/lib/python3.9/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_size_factor_key_expr' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
/opt/modules/i12g/anaconda/envs/scooby_reproducibility/lib/python3.9/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_size_factor_key_acc' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)


In [8]:
data_path = '/s/project/QNA/scborzoi/submission_data/'

In [10]:
with open(os.path.join(data_path, "motif_effects", "chromvar", "H12CORE_jaspar_format.txt"), 'r') as f:
    motifs = Bio.motifs.jaspar.read(f, format='jaspar')

In [14]:
adata = sc.read(
    os.path.join(data_path, 'bmmc_multiome_multivi_neurips21_curated_new_palantir_fixed.h5ad'))

In [15]:
adata = adata[:, adata.var.feature_types == 'ATAC'].copy()

In [16]:
adata.X = adata.layers['counts'].copy()

In [17]:
adata.X

<63683x116490 sparse matrix of type '<class 'numpy.float32'>'
	with 214287522 stored elements in Compressed Sparse Row format>

In [18]:
print((adata.X == 1).sum())
print((adata.X == 2).sum())

30446262
159924112


In [19]:
scvi.data.reads_to_fragments(adata)

In [20]:
adata.X = adata.layers['fragments'].copy()

In [21]:
print((adata.X == 1).sum())
print((adata.X == 2).sum())

190370374
20785981


In [22]:
adata = adata[adata.obs.l2_cell_type != 'Doublet']

In [23]:
sc.pp.filter_genes(adata, min_cells=int(adata.shape[0]*0.01))

/opt/modules/i12g/anaconda/envs/scooby_reproducibility/lib/python3.9/site-packages/scanpy/preprocessing/_simple.py:283: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var["n_cells"] = number


In [27]:
# identical contig names as used for alignment
pc.add_peak_seq(adata, genome_file=os.path.join(data_path,  "motif_effects", "chromvar", 'genome.fa'), delimiter="-")

100%|██████████| 61663/61663 [00:00<00:00, 94754.05it/s]


In [28]:
pc.add_gc_bias(adata)
pc.get_bg_peaks(adata)

100%|██████████| 61628/61628 [00:00<00:00, 125202.80it/s]


In [29]:
pc.match_motif(adata, motifs=motifs)

100%|██████████| 61628/61628 [07:46<00:00, 132.08it/s]


In [30]:
adata.write('adata_chromvar_tmp.h5ad')

In [3]:
adata = sc.read('adata_chromvar_tmp.h5ad')

In [6]:
dev = pc.compute_deviations(adata)
dev

2024-07-11 13:22:37 INFO     computing expectation reads per cell and peak...
2024-07-11 13:22:41 INFO     computing observed motif deviations...
2024-07-11 13:23:00 INFO     computing background deviations...
/opt/modules/i12g/anaconda/envs/scvi-tools-1/lib/python3.12/site-packages/anndata/_core/anndata.py:430: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(


AnnData object with n_obs × n_vars = 63788 × 1443

In [13]:
dev.write(os.path.join(data_path,  "motif_effects", "chromvar", 'deviations.h5ad'))

In [14]:
! rm adata_chromvar_tmp.h5ad